In [1]:
import pandas as pd
import numpy as np

In [2]:
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import pylab as plt

In [3]:
def clean_data():
    
    global db1
    db1 = db1[(db1.num_infections > 0)].reset_index()
    db1 = db1.drop(columns = ['index'], axis = 1)
    
    db1['date'] = pd.to_datetime(db1.date)
    
   
    db1['autonomous_region'] = db1.autonomous_region.apply(lambda x: 'Comunidad Valenciana' if x == 'Valenciana, Comunidad' else x)
    db1['autonomous_region'] = db1.autonomous_region.apply(lambda x: 'Comunidad de Madrid' if x == 'Madrid, Comunidad de' else x)
    db1['autonomous_region'] = db1.autonomous_region.apply(lambda x: 'Región de de Murcia' if x == 'Murcia, Región de' else x)
    db1['autonomous_region'] = db1.autonomous_region.apply(lambda x: 'Comunidad Foral de Navarra' if x == 'Navarra, Comunidad Foral de' else x)
    db1['autonomous_region'] = db1.autonomous_region.apply(lambda x: 'Principado de Asturias' if x == 'Asturias, Principado de' else x)
    
    
    db1['province'] = db1.province.apply(lambda x: 'Alicante' if x == 'Alicante/Alacant' else x)
    db1['province'] = db1.province.apply(lambda x: 'Castellón' if x == 'Castellón/Castelló' else x)
    db1['province'] = db1.province.apply(lambda x: 'Araba' if x == 'Araba/Álava' else x)
    
    db1['sex'] = db1.sex.apply(lambda x: 'UKNOWN' if x == 'NC' else x)
    
    db1['age_interval'] = db1.age_interval.apply(lambda x: 'UKNOWN' if x == 'NC' else x)

In [4]:
def cumulative(dataframe):
    
    cumu_num_infections = dataframe.num_infections.cumsum()
    cumu_num_hosp = dataframe.num_hosp.cumsum()
    cumu_num_uci = dataframe.num_uci.cumsum()
    cumu_num_dead = dataframe.num_dead.cumsum()

    cumulative = pd.DataFrame({'cumu_num_infections': cumu_num_infections, 'cumu_num_hosp': cumu_num_hosp, 
                               'cumu_num_uci': cumu_num_uci, 'cumu_num_dead': cumu_num_dead})
    dataframe = pd.concat([dataframe, cumulative], axis = 1)
    return dataframe

In [5]:
def freq_rel(dataframe):
    
    dataframe['Number of infection (%)'] = [round(i/ dataframe['num_infections'].sum(),3)*100 for i in dataframe['num_infections']]
    dataframe['Number of hospitalisation (%)'] = [round(i/ dataframe['num_hosp'].sum(),3)*100 for i in dataframe['num_hosp']]
    dataframe['Number of Intensive Care Unit (%)'] = [round(i/ dataframe['num_uci'].sum(),3)*100 for i in dataframe['num_uci']]
    dataframe['Number of deaths (%)'] = [round(i/ dataframe['num_dead'].sum(),3)*100 for i in dataframe['num_dead']]

In [6]:
def organise(column):
    
    name = db1.groupby(column).sum()
    freq_rel(name)
    
    return name

In [7]:
def mov_7_ave(dataframe):
    dataframe['ave_7_num_infections'] = dataframe.iloc[:,1].rolling(window=7).mean()
    dataframe['ave_7_num_hosp'] = dataframe.iloc[:,2].rolling(window=7).mean()
    dataframe['ave_7_num_uci'] = dataframe.iloc[:,3].rolling(window=7).mean()
    dataframe['ave_7_num_dead'] = dataframe.iloc[:,4].rolling(window=7).mean()
    
    return dataframe

In [8]:
db1 =pd.read_csv('/content/covid-19-sample.csv') 

In [9]:
clean_data()

In [10]:
bydate = db1.groupby('date').sum().reset_index()
bydate = cumulative(bydate)
bydate = mov_7_ave(bydate)

In [11]:
from statsmodels.tsa.arima_model import ARMA
from sklearn.metrics import mean_squared_error as mse

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [12]:
infections = bydate[['date', 'ave_7_num_infections']].set_index('date')
infections = infections[6:]

In [20]:
infections

,ave_7_num_infections
date,
2020-03-01,2.285714
2020-03-02,4.285714
2020-03-03,6.142857
2020-03-04,8.142857
2020-03-05,9.285714
...,...
2021-06-06,1057.000000
2021-06-07,1025.000000
2021-06-08,982.285714


In [18]:
res=[]
for i in range(1, len(infections)):
  train, test = infections[:-i], infections[-i:]
  for p in range(1, 3):
    for q in range(5, 8):
      model=ARMA(train, order = (p, q)).fit(disp=False)
      res.append((i, p, q, mse(model.predict(len(train), len(infections)-1), test)**0.5))

/usr/local/lib/python3.7/dist-packages/statsmodels/tsa/base/tsa_model.py:165: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  % freq, ValueWarning)
/usr/local/lib/python3.7/dist-packages/statsmodels/tsa/base/tsa_model.py:165: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  % freq, ValueWarning)
/usr/local/lib/python3.7/dist-packages/statsmodels/tsa/base/tsa_model.py:165: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  % freq, ValueWarning)
/usr/local/lib/python3.7/dist-packages/statsmodels/tsa/base/tsa_model.py:165: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  % freq, ValueWarning)
/usr/local/lib/python3.7/dist-packages/statsmodels/tsa/base/tsa_model.py:165: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  % freq, ValueWarning)
/usr/local/lib/python3.7/dist-packa

ValueError: ignored

In [19]:
res

[(1, 1, 5, 18.691732412702095),
 (1, 1, 6, 7.635324421145356),
 (1, 1, 7, 7.343326275215304),
 (1, 2, 5, 14.355866539294425),
 (1, 2, 6, 12.059105485006171),
 (1, 2, 7, 5.417555282340345),
 (2, 1, 5, 50.547178982584605),
 (2, 1, 6, 55.87796973322921),
 (2, 1, 7, 51.00849498317628),
 (2, 2, 5, 45.21354108800264),
 (2, 2, 6, 46.70732631021396),
 (2, 2, 7, 44.26772954529884),
 (3, 1, 5, 80.91580842883047),
 (3, 1, 6, 81.02487207887886),
 (3, 1, 7, 74.24133245610079),
 (3, 2, 5, 77.6444501513132),
 (3, 2, 6, 76.66434645208268),
 (3, 2, 7, 90.14882882926771),
 (4, 1, 5, 113.68862868977294),
 (4, 1, 6, 110.81582439563526),
 (4, 1, 7, 110.66600005441532),
 (4, 2, 5, 121.0374105854169),
 (4, 2, 6, 118.53350861519971),
 (4, 2, 7, 108.60288904229317),
 (5, 1, 5, 74.02064493007016),
 (5, 1, 6, 104.8569880825455),
 (5, 1, 7, 96.2762128996072),
 (5, 2, 5, 111.5375689123829),
 (5, 2, 6, 113.77238764236127),
 (5, 2, 7, 115.11096439355819),
 (6, 1, 5, 78.10907854355708),
 (6, 1, 6, 72.24115029130893),